Joining Timeseries
==================

One of the first and most important steps in comparing simulated and observed timeseries data is to join
the two datasets together based on location and time (and potentially other fields).

In this example, we consider a comparison of National Water Model (NWM) v3.0 retrospective
streamflow simulations ("secondary") to USGS observed streamflow data ("primary") for a few
time steps at five different locations (gage stations).

We'll use an example Evaluation dataset for this tutorial.

In [47]:
from teehr import Evaluation

ev = Evaluation("joining_tutorial_data")

**Primary Timeseries**: The USGS observed streamflow data at all locations and times.

In [48]:
ev.primary_timeseries.to_pandas()

,reference_time,value_time,value,unit_name,location_id,configuration_name,variable_name
0,NaT,1990-10-30 19:00:00,131.508148,m^3/s,usgs-01013500,usgs_observations,streamflow_daily_mean
1,NaT,1990-10-31 19:00:00,124.039581,m^3/s,usgs-01013500,usgs_observations,streamflow_daily_mean
2,NaT,1990-11-01 19:00:00,117.479515,m^3/s,usgs-01013500,usgs_observations,streamflow_daily_mean
3,NaT,1990-11-02 19:00:00,110.636276,m^3/s,usgs-01013500,usgs_observations,streamflow_daily_mean
4,NaT,1990-10-30 19:00:00,15.465717,m^3/s,usgs-01022500,usgs_observations,streamflow_daily_mean
5,NaT,1990-10-31 19:00:00,12.820452,m^3/s,usgs-01022500,usgs_observations,streamflow_daily_mean
6,NaT,1990-11-01 19:00:00,10.911425,m^3/s,usgs-01022500,usgs_observations,streamflow_daily_mean
7,NaT,1990-11-02 19:00:00,9.811788,m^3/s,usgs-01022500,usgs_observations,streamflow_daily_mean


**Secondary Timeseries**: The NWM v3.0 retrospective streamflow simulations at all locations and times.

In [49]:
ev.secondary_timeseries.to_pandas()

,reference_time,value_time,value,unit_name,location_id,configuration_name,variable_name
0,NaT,1990-10-30 19:00:00,30.135416,m^3/s,nwm30-2677104,nwm30_retrospective,streamflow_daily_mean
1,NaT,1990-10-31 19:00:00,26.719584,m^3/s,nwm30-2677104,nwm30_retrospective,streamflow_daily_mean
2,NaT,1990-11-01 19:00:00,23.416666,m^3/s,nwm30-2677104,nwm30_retrospective,streamflow_daily_mean
3,NaT,1990-11-02 19:00:00,20.728750,m^3/s,nwm30-2677104,nwm30_retrospective,streamflow_daily_mean
4,NaT,1990-10-30 19:00:00,96.385834,m^3/s,nwm30-724696,nwm30_retrospective,streamflow_daily_mean
5,NaT,1990-10-31 19:00:00,95.759163,m^3/s,nwm30-724696,nwm30_retrospective,streamflow_daily_mean
6,NaT,1990-11-01 19:00:00,94.721664,m^3/s,nwm30-724696,nwm30_retrospective,streamflow_daily_mean
7,NaT,1990-11-02 19:00:00,93.314583,m^3/s,nwm30-724696,nwm30_retrospective,streamflow_daily_mean


**Locations Crosswalks**: A mapping between the USGS and NWM location IDs

In [50]:
ev.location_crosswalks.to_pandas()

,primary_location_id,secondary_location_id
0,usgs-01013500,nwm30-724696
1,usgs-01022500,nwm30-2677104
2,usgs-01013500,usgs-01013500
3,usgs-01022500,usgs-01022500


**Location Geometry**: The point geometries of the USGS gage station locations.

In [52]:
ev.locations.to_geopandas()

,id,name,geometry
0,usgs-01013500,"Fish River near Fort Kent, Maine",POINT (-68.58278 47.23750)
1,usgs-01022500,"Narraguagus River at Cherryfield, Maine",POINT (-67.93528 44.60806)


**Attributes**: Additional information about each of the locations.

In [57]:
ev.location_attributes.to_pandas().head()

,location_id,attribute_name,value
0,usgs-01013500,slope_fdc,1.52821853538976
1,usgs-01022500,slope_fdc,1.77627980351081
2,usgs-01013500,soil_porosity,0.461148751156712
3,usgs-01022500,soil_porosity,0.415905486478906
4,usgs-01013500,pet_mean,1.97155451060917


There are a number of physical attributes associated with each location:

In [59]:
ev.location_attributes.to_pandas().attribute_name.unique()

array(['slope_fdc', 'soil_porosity', 'pet_mean', 'dom_land_cover_frac',
       'dom_land_cover', 'NID_dam_lengths', 'high_prec_freq',
       'ecoregion_L2', 'q_mean', 'q95', 'p_seasonality', 'q5',
       'runoff_ratio', 'aridity', 'baseflow_index', 'frac_snow', 'p_mean',
       'slope_mean', 'elev_mean', 'drainage_area', 'forest_frac',
       'high_q_freq', 'frac_urban', 'zero_q_freq', 'stream_order',
       'river_forecast_center'], dtype=object)

Ultimately, we want to combine all the data into a single table to facilitate efficient analysis and exploration based
on the location, time, and potentially some other attributes.  For example, we could start to ask questions like:
"How does the NWM model perform in primarily forested watersheds compared to primarily urban watersheds?"

First, we can join the primary and secondary timeseries by location and time without adding geometry or
attributes.  This requires the crosswalk table to map the primary and secondary location IDs. Because
the data may contain more than one variable (e.g., temperature, C) we also need to consider the ``variable_name``
and ``measurement_unit`` fields during the join.

:::{figure} ../../images/tutorials/joining/nwm_usgs_ex_joining_snip.png
---
height: 400px
width: 900px
---

Joining the primary and secondary streamflow values by location, time, variable name, and measurement unit.
:::

We can also join the location geometry and attributes to the joined timeseries table.  This will allow us to
easily filter and group the data based on the location attributes, and to visualize the output.

To join the geometry, we can simply map each primary location ID in the joined timeseries table to the ID in the
geometry table, which in this case contains the point geometries of the USGS gage stations.

:::{figure} ../../images/tutorials/joining/nwm_usgs_ex_joining_geometry.png
---
height: 400px
width: 900px
---

Joining the geometry to the initial joined timeseries table.
:::

Finally, we can join additional, pre-calculated attributes the table, which give us more options for
filtering and grouping the data when calculating performance metrics.

:::{figure} ../../images/tutorials/joining/nwm_usgs_ex_joining_attributes.png
---
height: 400px
width: 900px
---

Joining the attributes to the initial joined timeseries table.
:::

TEEHR can automatically join the timeseries and location attributes, while simulaneously running user-defined functions on specified fields.

In [64]:
ev.joined_timeseries.create(execute_udf=True)

In [62]:
ev.joined_timeseries.to_pandas()

,reference_time,value_time,primary_location_id,secondary_location_id,primary_value,secondary_value,unit_name,location_id,soil_porosity,dom_land_cover_frac,...,river_forecast_center,month,year,water_year,primary_normalized_flow,secondary_normalized_flow,over_90th_pct,over_90th_pct_event_id,configuration_name,variable_name
0,NaT,1990-10-30 19:00:00,usgs-01013500,nwm30-724696,131.508148,96.385834,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,NERFC,10,1990,1991,0.058161,0.042628,True,1990-10-30 19:00:00-1990-10-30 19:00:00,nwm30_retrospective,streamflow_daily_mean
1,NaT,1990-10-31 19:00:00,usgs-01013500,nwm30-724696,124.039581,95.759163,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,NERFC,10,1990,1991,0.054858,0.042351,False,None,nwm30_retrospective,streamflow_daily_mean
2,NaT,1990-11-01 19:00:00,usgs-01013500,nwm30-724696,117.479515,94.721664,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,NERFC,11,1990,1991,0.051957,0.041892,False,None,nwm30_retrospective,streamflow_daily_mean
3,NaT,1990-11-02 19:00:00,usgs-01013500,nwm30-724696,110.636276,93.314583,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,NERFC,11,1990,1991,0.048930,0.041270,False,None,nwm30_retrospective,streamflow_daily_mean
4,NaT,1990-10-30 19:00:00,usgs-01022500,nwm30-2677104,15.465717,30.135416,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,NERFC,10,1990,1991,0.026307,0.051259,True,1990-10-30 19:00:00-1990-10-30 19:00:00,nwm30_retrospective,streamflow_daily_mean
5,NaT,1990-10-31 19:00:00,usgs-01022500,nwm30-2677104,12.820452,26.719584,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,NERFC,10,1990,1991,0.021807,0.045449,False,None,nwm30_retrospective,streamflow_daily_mean
6,NaT,1990-11-01 19:00:00,usgs-01022500,nwm30-2677104,10.911425,23.416666,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,NERFC,11,1990,1991,0.018560,0.039831,False,None,nwm30_retrospective,streamflow_daily_mean
7,NaT,1990-11-02 19:00:00,usgs-01022500,nwm30-2677104,9.811788,20.728750,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,NERFC,11,1990,1991,0.016690,0.035259,False,None,nwm30_retrospective,streamflow_daily_mean


The geometry data is joined on the fly when a ``GeoDataFrame`` is requested.

In [65]:
ev.joined_timeseries.to_geopandas()

,primary_location_id,reference_time,value_time,secondary_location_id,primary_value,secondary_value,unit_name,location_id,soil_porosity,dom_land_cover_frac,...,month,year,water_year,primary_normalized_flow,secondary_normalized_flow,over_90th_pct,over_90th_pct_event_id,configuration_name,variable_name,geometry
0,usgs-01013500,NaT,1990-10-30 19:00:00,nwm30-724696,131.508148,96.385834,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,10,1990,1991,0.058161,0.042628,True,1990-10-30 19:00:00-1990-10-30 19:00:00,nwm30_retrospective,streamflow_daily_mean,POINT (-68.58278 47.23750)
1,usgs-01013500,NaT,1990-10-31 19:00:00,nwm30-724696,124.039581,95.759163,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,10,1990,1991,0.054858,0.042351,False,None,nwm30_retrospective,streamflow_daily_mean,POINT (-68.58278 47.23750)
2,usgs-01013500,NaT,1990-11-01 19:00:00,nwm30-724696,117.479515,94.721664,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,11,1990,1991,0.051957,0.041892,False,None,nwm30_retrospective,streamflow_daily_mean,POINT (-68.58278 47.23750)
3,usgs-01013500,NaT,1990-11-02 19:00:00,nwm30-724696,110.636276,93.314583,m^3/s,usgs-01013500,0.461148751156712,0.883451918625571,...,11,1990,1991,0.048930,0.041270,False,None,nwm30_retrospective,streamflow_daily_mean,POINT (-68.58278 47.23750)
4,usgs-01022500,NaT,1990-10-30 19:00:00,nwm30-2677104,15.465717,30.135416,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,10,1990,1991,0.026307,0.051259,True,1990-10-30 19:00:00-1990-10-30 19:00:00,nwm30_retrospective,streamflow_daily_mean,POINT (-67.93528 44.60806)
5,usgs-01022500,NaT,1990-10-31 19:00:00,nwm30-2677104,12.820452,26.719584,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,10,1990,1991,0.021807,0.045449,False,None,nwm30_retrospective,streamflow_daily_mean,POINT (-67.93528 44.60806)
6,usgs-01022500,NaT,1990-11-01 19:00:00,nwm30-2677104,10.911425,23.416666,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,11,1990,1991,0.018560,0.039831,False,None,nwm30_retrospective,streamflow_daily_mean,POINT (-67.93528 44.60806)
7,usgs-01022500,NaT,1990-11-02 19:00:00,nwm30-2677104,9.811788,20.728750,m^3/s,usgs-01022500,0.415905486478906,0.820493445115513,...,11,1990,1991,0.016690,0.035259,False,None,nwm30_retrospective,streamflow_daily_mean,POINT (-67.93528 44.60806)


Now that the data is joined into a single table, we can easily filter and groupby the available fields to pre-calculated
performance metrics, such as the Nash-Sutcliffe Efficiency (NSE) or the Kling-Gupta Efficiency (KGE), and create visualizations.